In [1]:
# PTT title generator from content base on seq2seq model.
import pandas as pd
import time
import csv
import os
import types 

#Initialize for files name and path.
base_dir = 'C:/Users/User/Raw data/PTT'
totalboard_name = 'TJ-BG-AT-SX-CC-MV'
board_name = ['Tech_job','Boy-Girl','AllTogether','sex','C_Chat','movie']
board_dict_index = {'Tech_job':0,'Boy-Girl':1,'AllTogether':2,'sex':3,'C_Chat':4,'movie':5}
max_word_length = 256
min_word_length = 100
max_title_length = 20
label_newnum = len(board_name)

new_content_dir = []
totalboard_dir = os.path.join(base_dir, totalboard_name)
if not os.path.exists(totalboard_dir):
    os.makedirs(totalboard_dir)
for name_ind in range(len(board_name)):
    # New data path
    new_content_dir.append(os.path.join(totalboard_dir, board_name[name_ind]+'_content'))
    if not os.path.exists(new_content_dir[name_ind]):
        os.makedirs(new_content_dir[name_ind])

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import jieba

# Import dictionary fron jieba and PTT.
jieba.set_dictionary('C:/Users/User/Anaconda3/Lib/site-packages/jieba/dict.txt.big')
jieba.load_userdict('C:/Users/User/Anaconda3/Lib/site-packages/jieba/userdict.txt')

text_content = []
text_label = []
text_title = []

GO_word = '<GO>' #解碼器端的句子起始標識符。
EOS_word = '<EOS>' #解碼器端的句子結束標識符。
PAD_word = '<PAD>' #補全字符。
UNK_word = '<UNK>' #低頻詞或者一些未遇到過的詞等。

# Load PTT content and information.
#dfs_total = pd.read_csv(os.path.join(totalboard_dir, totalboard_name + '-test.csv'))
dfs_total = pd.read_csv(os.path.join(totalboard_dir, totalboard_name + '.csv'))
len_total = len(dfs_total)

for dfs_index in range(len_total):
    read_index = int(dfs_total.iloc[dfs_index, 0])
    type_name = str(dfs_total.iloc[dfs_index, 1])
    word_length = int(dfs_total.iloc[dfs_index, 5])
    title_name = str(dfs_total.iloc[dfs_index, 9])
    this_board_name = str(dfs_total.iloc[dfs_index, 10])
    
    # Add start and end character.
    text = GO_word + ' '
    with open(os.path.join(new_content_dir[board_dict_index[this_board_name]], str(read_index) + '.csv'),
                'r', encoding = 'utf-8-sig') as file:
        csvCursor = csv.reader(file)
        for rows in csvCursor:
            for row in rows:
                # Read content and remove empty.
                text = text + row
        else:
            text = text + ' ' + EOS_word
    
    # Split titles to words by Jieba.
    title_name.encode('utf-8-sig')
    title_jieba = jieba.cut(title_name, cut_all=False)
    title_name = GO_word
    word_length = 0
    for word in title_jieba:
        word_length += 1
        title_name = title_name + ' ' + word
    else:
        title_name = title_name + ' ' + EOS_word

    text_content.append(text)
    text_label.append(board_dict_index[this_board_name])
    text_title.append(title_name)
    file.close()

# Build vocabulary and convert content to sequence by Keras tool.
words_limit = 60000
tokenizer = Tokenizer(num_words=words_limit, 
                      filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n　，。！：；、？﹝﹞「」『』（）｛｝［］【】《》“”‘’＼｜〝〞‵′＋－＊／＝≦≧＿＠＃＄％︿＆～§◎．※ㄧ↔│○●☆★◇◆□■▽▼△▲㊣⊙⊕ˍ…﹌﹋﹎﹍﹉﹊‥–↑↓←→↖↗↙↘∥∕℅≒≡∩∪∞￣＿◤◥◣◢∵∴〒⊥∠⊿┼┴┬┤├▔─│▕┌┐└┘╭╮╰╯═╞╪╡╔╦╗╠╬╣╚╩╝╒╤╕╘╧╛╓╥╖╟╫╢╙╨╜║▓╱╲╳▁▂▄▅▆▇█▏▎▍▌▋▊▉▁▔', 
                      lower=False,
                      split=" ")
                      #oov_token=UNK_word)
tokenizer.fit_on_texts(text_content+text_title)
vocab = tokenizer.word_index
vocab_counts = tokenizer.word_counts
vocab_docs = tokenizer.word_docs

print(len(vocab))
anti_vocab = {}
vocab_keys = list(vocab.keys())
for vocab_word in vocab:
    vocab_index = vocab[vocab_word]
    anti_vocab[vocab_index] = vocab_keys[vocab_index-1]
    if vocab_index % 1000 == 0 or vocab_index < 20:
        print(vocab[vocab_word], vocab_word, anti_vocab[vocab_index], vocab_counts[vocab_word], vocab_docs[vocab_word])

from sklearn.cross_validation import train_test_split
from keras.utils import np_utils

x_train, x_test, y_train, y_test = train_test_split(text_content, text_title, test_size=0.2, random_state=9487)

# 将每个词用词典中的数值代替
x_train_word_ids = tokenizer.texts_to_sequences(x_train)
x_test_word_ids = tokenizer.texts_to_sequences(x_test)
y_train_word_ids = tokenizer.texts_to_sequences(y_train)
y_test_word_ids = tokenizer.texts_to_sequences(y_test)
# 序列模式
x_train = pad_sequences(x_train_word_ids, maxlen=max_word_length+2, padding='post', truncating='post')
x_test = pad_sequences(x_test_word_ids, maxlen=max_word_length+2, padding='post', truncating='post')
y_train = pad_sequences(y_train_word_ids, maxlen=max_title_length+2, padding='post', truncating='post')
y_test = pad_sequences(y_test_word_ids, maxlen=max_title_length+2, padding='post', truncating='post')
# 序列長度
x_train_size = [len(word_ids) for word_ids in x_train_word_ids]
x_test_size = [len(word_ids) for word_ids in x_test_word_ids]
y_train_size = [len(word_ids) for word_ids in y_train_word_ids]
y_test_size = [len(word_ids) for word_ids in y_test_word_ids]

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Building prefix dict from C:\Users\User\Anaconda3\Lib\site-packages\jieba\dict.txt.big ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.ufa6ae29b0cbce8b45e006c7fa30eaaf8.cache
Loading model cost 0.959 seconds.
Prefix dict has been built succesfully.


199446
1 的 的 479528 108422
2 我 我 187622 60294
3 <GO> <GO> 172800 172800
4 <EOS> <EOS> 172800 172800
5 是 是 157302 68972
6 了 了 129931 60543
7 有 有 108041 59614
8 在 在 97066 54361
9 也 也 86587 48759
10 都 都 75404 45560
11 就 就 71400 42410
12 不 不 66238 42797
13 很 很 54725 34820
14 會 會 54210 36201
15 他 他 49560 20170
16 跟 跟 48539 31439
17 看 看 48476 30452
18 你 你 48168 21707
19 想 想 47770 34168
1000 跳 跳 1246 1022
2000 整合 整合 607 489
3000 往往 往往 366 345
4000 007 007 255 156
5000 處於 處於 192 188
6000 一般來說 一般來說 154 150
7000 何 何 125 101
8000 有鬼 有鬼 105 100
9000 備 備 89 89
10000 半年前 半年前 78 78
11000 掉下來 掉下來 68 66
12000 托爾 托爾 61 46
13000 傻子 傻子 54 51
14000 活得 活得 49 48
15000 寫給 寫給 44 41
16000 希斯 希斯 41 31
17000 毆打 毆打 37 33
18000 編制 編制 34 29
19000 激怒 激怒 32 30
20000 神秘性 神秘性 30 5
21000 睡意 睡意 27 27
22000 真情 真情 26 24
23000 寫些 寫些 24 24
24000 野蠻女友 野蠻女友 23 14
25000 20th 20th 21 17
26000 世家 世家 20 20
27000 天降 天降 19 18
28000 人卡 人卡 18 13
29000 決定權 決定權 17 17
30000 始於 始於 16 15
31000 初號機 初號機 16 10
32000 早睡早起 早睡早起 15 15
33000 記號 記號

C:\Users\User\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


 2 2
115000 heartdrunken heartdrunken 2 2
116000 而妮 而妮 2 2
117000 降職 降職 2 1
118000 弄手 弄手 2 2
119000 跟織 跟織 2 2
120000 那聲 那聲 2 2
121000 臭氣沖天 臭氣沖天 2 2
122000 約聘職 約聘職 2 2
123000 亞伯特 亞伯特 2 2
124000 母子之情 母子之情 2 2
125000 一撥 一撥 2 2
126000 foreigner foreigner 2 2
127000 沒兵 沒兵 2 2
128000 速到 速到 2 2
129000 119999999119999991199911999999911991199 119999999119999991199911999999911991199 2 2
130000 愛主 愛主 2 2
131000 業務經理 業務經理 2 2
132000 舒涵 舒涵 2 2
133000 浮台 浮台 2 1
134000 和吳 和吳 2 2
135000 Gatebox Gatebox 2 1
136000 whatthefack whatthefack 2 2
137000 邊默數 邊默數 2 2
138000 李芳 李芳 2 1
139000 群后 群后 2 2
140000 找外 找外 2 2
141000 防空壕 防空壕 2 2
142000 道德哲學 道德哲學 1 1
143000 Footlight Footlight 1 1
144000 Ds Ds 1 1
145000 全廢 全廢 1 1
146000 喪夫 喪夫 1 1
147000 櫻看 櫻看 1 1
148000 DOCtRINE DOCtRINE 1 1
149000 曼今 曼今 1 1
150000 NanoDesu NanoDesu 1 1
151000 不佳來 不佳來 1 1
152000 重禮 重禮 1 1
153000 帶冷 帶冷 1 1
154000 繳為 繳為 1 1
155000 乃讀 乃讀 1 1
156000 附好 附好 1 1
157000 無病 無病 1 1
158000 1JPmnQcb 1JPmnQcb 1 1
159000 他博 他博 1 1
160000 歸頭 歸頭 1 1
1

In [12]:
import tensorflow as tf
################
# Build seq2seq model
################
def build_model_in():
    ######## model input tensor ########
    is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    input_seq_len = tf.placeholder(tf.int32, (None,), name='input_seq_len')
    target_seq_len = tf.placeholder(tf.int32, (None,), name='target_seq_len')
    limit_target_seq_len = tf.placeholder(tf.int32, (), name='limit_target_seq_len')
    return is_training, inputs, targets, input_seq_len, target_seq_len, limit_target_seq_len
    
def build_model(is_training, inputs, targets, input_seq_len, target_seq_len, limit_target_seq_len):
    max_target_seq_len = tf.reduce_max(target_seq_len, name='max_target_seq_len')
    validation_batch_size = tf.shape(targets)[0]
    start_tokens = tf.tile(tf.constant([vocab[GO_word]], dtype=tf.int32),
                               [validation_batch_size], name='start_tokens')
    
    ######## Embedding for encoder and decoder ########
    embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size]))
    encoder_embed_input = tf.nn.embedding_lookup(embeddings, inputs)
    decoder_embed_input = tf.nn.embedding_lookup(embeddings, targets)
    
    ######## Encoder by LSTM ########
    # LSTM cell
    def get_lstm_cell(rnn_size, layer):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size, 
                                            initializer=tf.random_uniform_initializer(-0.1, 0.1))
#activity_regularizer=tf.contrib.layers.l2_regularizer(l2_reg_const) 
        if is_training is not None and lstm_dropout < 1.0:
            if layer == 0:
                lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, 
                                                          input_keep_prob=lstm_dropout, 
                                                          output_keep_prob=lstm_dropout)
            else:
                lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, 
                                                          output_keep_prob=lstm_dropout)
        return lstm_cell

    # Struct encoder by LSTM bi-directional multi-layer
    encoder_f_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size, layer) for layer in range(num_layers)])
    encoder_b_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size, layer) for layer in range(num_layers)])
    encoder_bi_outputs, encoder_bi_state = tf.nn.bidirectional_dynamic_rnn(encoder_f_cell, 
                                                                           encoder_b_cell, 
                                                                           encoder_embed_input, 
                                                                           sequence_length=input_seq_len, 
                                                                           dtype=tf.float32)
    encoder_outputs = tf.concat((encoder_bi_outputs[0], encoder_bi_outputs[1]), 2)
    encoder_state_c = [tf.concat((state[0].c, state[1].c), 1) for state in encoder_bi_state]
    encoder_state_h = [tf.concat((state[0].h, state[1].h), 1) for state in encoder_bi_state]
    encoder_state = tf.contrib.rnn.LSTMStateTuple(c=encoder_state_c, h=encoder_state_h)
    
    ######## Decoder by LSTM ########
    # Bahdanau attention
    def get_decoder_cell(rnn_size, layer, attn_layers, encoder_outputs, input_seq_len):
        #attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(rnn_size*2, encoder_outputs,
        attention_mechanism = tf.contrib.seq2seq.LuongAttention(rnn_size*2, encoder_outputs,
                                                                memory_sequence_length=input_seq_len)
        attn_cell = tf.contrib.seq2seq.AttentionWrapper(get_lstm_cell(rnn_size*2, layer), 
                                                       attention_mechanism, 
                                                       attention_layer_size=attn_layers)
        #decoder_cell = tf.contrib.rnn.OutputProjectionWrapper(attn_cell, slot_size)
        return attn_cell
    
    # Struct LSTM cell in Decoder
    decoder_cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell(rnn_size, layer, num_layers, encoder_outputs, input_seq_len) for layer in range(num_layers)])
    
    # Output fully-connecting layer
    output_layer = Dense(vocab_size, 
                         kernel_initializer=tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    # Struct training decoder
    with tf.variable_scope('decode'):
        training_helper = tf.contrib.seq2seq.TrainingHelper(decoder_embed_input,
                                                            target_seq_len)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, training_helper,
                                                           decoder_cell.zero_state(dtype=tf.float32, batch_size=batch_size), 
                                                           output_layer) 
        training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                           impute_finished=True,
                                                                           maximum_iterations=max_target_seq_len)
    # Struct predicting decoder (Share variable with training)
    with tf.variable_scope('decode', reuse=True):
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                     start_tokens, 
                                                                     vocab[EOS_word])
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, predicting_helper,
                                                             decoder_cell.zero_state(dtype=tf.float32, batch_size=batch_size), 
                                                             output_layer)
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(predicting_decoder,
                                                                            impute_finished=True,
                                                                            maximum_iterations=limit_target_seq_len)
    ######## Model output tensor ########
    training_logits = tf.identity(training_decoder_output.rnn_output, name='trainings')
    predicting_logits = tf.identity(predicting_decoder_output.sample_id, name='predictions')
    return training_logits, predicting_logits, max_target_seq_len
    
def build_optimizer(training_logits, predicting_logits, targets, target_seq_len, max_target_seq_len):
    ######## Optimization ########
    if is_training is not None:
        with tf.name_scope('optimization'):
            masks = tf.sequence_mask(target_seq_len, max_target_seq_len, dtype=tf.float32)
            global_step = tf.Variable(0, trainable=False)
            l2 = tf.Variable(0.0, trainable=False)
            # Loss function
            cost = tf.contrib.seq2seq.sequence_loss(training_logits, targets, masks, name='sequence_cost')
            # Add regularization to Loss
            l2 = l2_reg_const * sum( tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables() if not ('Bias' in tf_var.name))
            #regularization_loss = tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
            cost = cost + l2
            # Optimizer
            learning_rate = tf.train.cosine_decay(lr, global_step, decay_steps, alpha=lr_alpha)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            
            # Gradient Clipping
            clip_rate = tf.train.cosine_decay(clip_const, global_step, decay_steps, alpha=clip_alpha)
            gradients, vriables = zip(*optimizer.compute_gradients(cost))
            gradients = [None if gradient is None else tf.clip_by_norm(gradient, clip_rate) for gradient in gradients]
            global_step += 1
            train_step = optimizer.apply_gradients(zip(gradients, vriables), name='train_step')
    return cost, train_step

In [13]:
import os
import time
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.contrib import layers

################
# Set parameter
################
patience = 0
num_epoch = 20
batch_size = 64
test_batch_size = 64
vocab_size = words_limit
limit_generation_len = max_title_length + 2
num_instances = len(y_train)
iter_per_epoch = int(num_instances / batch_size)
test_num_instances = len(y_test)
test_iter_per_epoch = int(test_num_instances / test_batch_size)

rnn_size = 150
num_layers = 3
slot_size = 172
embedding_size = 300

lstm_dropout = 0.5
l2_reg_const = 0.001

lr = 0.01
lr_alpha = 1e-3
decay_steps = 10 * num_instances / batch_size

clip_const = 50.0
clip_alpha = 5.0

total_start_t = time.time()
build_new_model = 1
if build_new_model:
    str_epoch = 0
    model_name = 'PTT_Encoder-Decoder_model_fixed_' + str(total_start_t)
else:
    str_epoch = 30
    model_name = 'PTT_Encoder-Decoder_model_fixed_1522421878.8546176'

################
# Prepare to run session
################
iter_pct10 = int(iter_per_epoch / 10)
if (num_instances % batch_size) > 0:
    iter_per_epoch += 1
batch_cutoff = [0]
for i in range(iter_per_epoch - 1):
    batch_cutoff.append(batch_size * (i+1))
batch_cutoff.append(num_instances)

best_validation_loss = 0.0
early_stop_counter = 0

mdl_dir = os.path.join(base_dir, 'model')
if not os.path.exists(mdl_dir):
    os.makedirs(mdl_dir)
model_dir = os.path.join(mdl_dir, model_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

################
# Run model session
################
train_graph = tf.Graph()
if build_new_model:
    ######## Build model parameters ########
    with train_graph.as_default():
        # Build input parameters
        is_training, inputs, targets, input_seq_len, target_seq_len, limit_target_seq_len = build_model_in()
        # Build model and  weight parameters
        training_logits, predicting_logits, max_target_seq_len = build_model(is_training, inputs, targets, 
                                                                             input_seq_len, target_seq_len, 
                                                                             limit_target_seq_len)
        # Build optimizer
        cost, train_step = build_optimizer(training_logits, predicting_logits,
                                           targets, target_seq_len, max_target_seq_len)

with tf.Session(graph=train_graph) as sess:
    if build_new_model:
        sess.run(tf.global_variables_initializer())
        best_epoch = 1
    else:
        ######## Load model parameters ########
        # Load model and weight parameters
        best_epoch = str_epoch
        load_model_dir = os.path.join(model_dir, 'model-{}'.format(str_epoch))
        loader = tf.train.import_meta_graph(load_model_dir + '.meta')
        loader.restore(sess, load_model_dir)
        # Load named input parameters
        is_training = train_graph.get_tensor_by_name('is_training:0')
        inputs = train_graph.get_tensor_by_name('inputs:0')
        targets = train_graph.get_tensor_by_name('targets:0')
        input_seq_len = train_graph.get_tensor_by_name('input_seq_len:0')
        target_seq_len = train_graph.get_tensor_by_name('target_seq_len:0')
        limit_target_seq_len = train_graph.get_tensor_by_name('limit_target_seq_len:0')
        # Load named output and optimizer parameters
        training_logits = train_graph.get_tensor_by_name('trainings:0')
        predicting_logits = train_graph.get_tensor_by_name('predictions:0')
        cost = train_graph.get_tensor_by_name('optimization/sequence_cost/truediv:0')
        train_step = train_graph.get_operation_by_name('optimization/gradients_1')
        
    ######## Run model epoch ########
    for e in range(str_epoch,str_epoch+num_epoch):
        rand_idxs = np.random.permutation(num_instances)
        print('\n========')
        print('epoch' + str(e+1))
        print('========')
        start_t = time.time()
        
        ######## Get random input batch and training model. ########
        for i in range(iter_per_epoch):
            X_batch = []
            Y_batch = []
            X_size = []
            Y_size = []
            for n in range(batch_cutoff[i],batch_cutoff[i+1]):
                X_batch.append(x_train[rand_idxs[n]])
                Y_batch.append(y_train[rand_idxs[n]])
                X_size.append(x_train_size[rand_idxs[n]])
                Y_size.append(y_train_size[rand_idxs[n]])
            max_X_size = max(X_size)
            max_Y_size = max(Y_size)
            X_batch = np.array(X_batch)
            Y_batch = np.array(Y_batch)
            X_size = np.array(X_size)
            Y_size = np.array(Y_size)
            
            _, loss = sess.run( [train_step, cost],
                                {is_training: True,
                                 inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size],
                                 input_seq_len: X_size, target_seq_len: Y_size,
                                 limit_target_seq_len: limit_generation_len})
            
            if i % iter_pct10 == iter_pct10 - 1:
                print('Iteration ',i+1,' (',round(i*100/iter_per_epoch,2),'%) Train Loss: ',
                      loss,'; run time[s]: ', time.time() - start_t,end='\n')
        
        print('Train Loss: ',loss)
        
        ######## Get validation input batch and validating model. ########
        test_loss = []
        for i in range(test_iter_per_epoch):
            i_str = i * test_batch_size
            i_end = i_str + test_batch_size
            
            X_batch = np.array(x_test[i_str:i_end])
            Y_batch = np.array(y_test[i_str:i_end])
            X_size = np.array(x_test_size[i_str:i_end])
            Y_size = np.array(y_test_size[i_str:i_end])
            max_X_size = max(X_size)
            max_Y_size = max(Y_size)
            y_loss = sess.run( cost,{inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size],
                                     input_seq_len: X_size, target_seq_len: Y_size,
                                     limit_target_seq_len: limit_generation_len})
            test_loss.append(y_loss)
        else:
            test_loss = np.asarray(test_loss)
            validation_loss = test_loss.mean()
            validation_loss_std = test_loss.std()
        print('Test loss: ',validation_loss,'; Std: ',validation_loss_std)
        
        ######## Generate title form testing content and validating model. ########
        answer_logits = sess.run( predicting_logits, 
                                {inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size], 
                                 input_seq_len: X_size, target_seq_len: Y_size,
                                 limit_target_seq_len: limit_generation_len})
        output_index = random.randint(0,test_batch_size-1)
        print('Target Words: {}'.format(" ".join([anti_vocab[i] for i in Y_batch[output_index] if i != 0])))
        print('Response Words: {}'.format(" ".join([anti_vocab[i] for i in answer_logits[output_index] if i != 0])))
        
        ######## Save model weight parameters and information. ########
        if validation_loss < best_validation_loss or e == 0:
            best_validation_loss = validation_loss
            best_epoch = e
            early_stop_counter = 0
            print('Save best score!! '+str(best_validation_loss))
        else:
            early_stop_counter += 1
            print('Count early stop!! '+str(early_stop_counter))

        print('Elapsed time in epoch ' + str(e+1) + ': ' + str(time.time() - start_t) + ' [s]')

        model_path = os.path.join(model_dir, 'model-%d' %(e+1))
        saver = tf.train.Saver()
        saver.save(sess, model_path)
        
        if patience != 0 and early_stop_counter >= patience:
            print('\n#######')
            print('Best model')
            print('#######')
            print('Stop by early stopping')
            print('Best score: ', best_validation_loss, 'Beat model: ', best_epoch)
            break

print('\nElapsed time in total: ' + str(time.time() - total_start_t))


epoch1
Iteration  108  ( 9.91 %) Train Loss:  43.304737 ; run time[s]:  81.00932288169861
Iteration  216  ( 19.91 %) Train Loss:  6.895974 ; run time[s]:  159.58211660385132
Iteration  324  ( 29.91 %) Train Loss:  6.8805294 ; run time[s]:  237.93737936019897
Iteration  432  ( 39.91 %) Train Loss:  6.860211 ; run time[s]:  316.9303369522095
Iteration  540  ( 49.91 %) Train Loss:  7.1416106 ; run time[s]:  395.0920808315277
Iteration  648  ( 59.91 %) Train Loss:  7.6475425 ; run time[s]:  473.487446308136
Iteration  756  ( 69.91 %) Train Loss:  6.918943 ; run time[s]:  552.2391877174377
Iteration  864  ( 79.91 %) Train Loss:  6.866954 ; run time[s]:  629.6830413341522
Iteration  972  ( 89.91 %) Train Loss:  6.531621 ; run time[s]:  707.0747621059418
Iteration  1080  ( 99.91 %) Train Loss:  6.537481 ; run time[s]:  784.6339077949524
Train Loss:  6.537481
Test loss:  6.620248 ; Std:  0.14864811
Target Words: <GO> 白台 是不是 一天到晚 都 盯 著 別人 褲檔 看 <EOS>
Response Words: <GO> 我 的 的 <EOS>
Save best s

Iteration  972  ( 89.91 %) Train Loss:  6.5180817 ; run time[s]:  688.6385669708252
Iteration  1080  ( 99.91 %) Train Loss:  6.625304 ; run time[s]:  765.1890280246735
Train Loss:  6.625304
Test loss:  6.6070476 ; Std:  0.12851271
Target Words: <GO> 南科 包子 TFT 製程 設備 工程師 <EOS>
Response Words: <GO> 關於 的 <EOS>
Count early stop!! 6
Elapsed time in epoch 8: 835.3496370315552 [s]

epoch9
Iteration  108  ( 9.91 %) Train Loss:  7.2468357 ; run time[s]:  76.20155620574951
Iteration  216  ( 19.91 %) Train Loss:  6.708917 ; run time[s]:  152.6437737941742
Iteration  324  ( 29.91 %) Train Loss:  6.7817893 ; run time[s]:  229.07994484901428
Iteration  432  ( 39.91 %) Train Loss:  6.6239 ; run time[s]:  305.4138481616974
Iteration  540  ( 49.91 %) Train Loss:  6.788093 ; run time[s]:  382.1518213748932
Iteration  648  ( 59.91 %) Train Loss:  6.4652553 ; run time[s]:  458.6451930999756
Iteration  756  ( 69.91 %) Train Loss:  6.850077 ; run time[s]:  535.2227439880371
Iteration  864  ( 79.91 %) Train L

Iteration  540  ( 49.91 %) Train Loss:  6.6754956 ; run time[s]:  384.0308048725128
Iteration  648  ( 59.91 %) Train Loss:  6.3921556 ; run time[s]:  460.73768973350525
Iteration  756  ( 69.91 %) Train Loss:  6.5141697 ; run time[s]:  536.9051430225372
Iteration  864  ( 79.91 %) Train Loss:  6.7466755 ; run time[s]:  613.0304877758026
Iteration  972  ( 89.91 %) Train Loss:  6.4498606 ; run time[s]:  690.0743005275726
Iteration  1080  ( 99.91 %) Train Loss:  6.592837 ; run time[s]:  766.7611410617828
Train Loss:  6.592837
Test loss:  6.475089 ; Std:  0.15730193
Target Words: <GO> 注意 我 甜 竟 幫 這個 男 的 慶生 <EOS>
Response Words: <GO> 請問 喜歡 我 我 我 女生 女生 女生 片 片 喜歡 女生 女生 女生 女生 女生 電影 的 電影 的 的
Save best score!! 6.475089
Elapsed time in epoch 16: 837.1032180786133 [s]

epoch17
Iteration  108  ( 9.91 %) Train Loss:  6.1966934 ; run time[s]:  76.61464977264404
Iteration  216  ( 19.91 %) Train Loss:  6.8668995 ; run time[s]:  153.10093998908997
Iteration  324  ( 29.91 %) Train Loss:  6.7407303 ; run tim

In [14]:
import os
import random
################################
#Generate title form testing content and validating model. 
################################
base_dir = 'C:/Users/User/Raw data/PTT'
model_name = 'PTT_Encoder-Decoder_model_fixed_1522514424.640014'
epoch = 20
model_path = os.path.join(base_dir, 'model', model_name, 'model-{}'.format(epoch))
print('Test epoch: ', epoch)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load model
    loader = tf.train.import_meta_graph(model_path + '.meta')
    loader.restore(sess, model_path)
    # Named parameters
    #is_training = loaded_graph.get_tensor_by_name('is_training:0')
    inputs = loaded_graph.get_tensor_by_name('inputs:0')
    targets = loaded_graph.get_tensor_by_name('targets:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    input_seq_len = loaded_graph.get_tensor_by_name('input_seq_len:0')
    target_seq_len = loaded_graph.get_tensor_by_name('target_seq_len:0')
    limit_target_seq_len = loaded_graph.get_tensor_by_name('limit_target_seq_len:0')
    
    for _ in range(8):
        i_str = random.randint(0,test_num_instances-1)
        i_end = i_str + 1

        X_batch = np.tile(x_test[i_str], (test_batch_size, 1))
        Y_batch = np.tile(y_test[i_str], (test_batch_size, 1))
        X_size = np.tile(x_test_size[i_str], (test_batch_size))
        Y_size = np.tile(y_test_size[i_str], (test_batch_size))
        max_X_size = x_test_size[i_str]
        max_Y_size = y_test_size[i_str]
        answer_logits = sess.run( logits, {inputs: X_batch, targets: Y_batch, 
                                             input_seq_len: X_size, target_seq_len: Y_size,
                                             limit_target_seq_len: limit_generation_len})

        output_index = 0 #random.randint(0,test_batch_size-1)
        print('\n========')
        print('Test index: ', i_str + output_index)
        print('========')
        print('Test input Words: {}'.format(" ".join([anti_vocab[i] for i in X_batch[output_index] if i != 0])))
        print('Target Words: {}'.format(" ".join([anti_vocab[i] for i in Y_batch[output_index] if i != 0])))
        print('Response Words: {}'.format(" ".join([anti_vocab[i] for i in answer_logits[output_index] if i != 0])))

Test epoch:  20
INFO:tensorflow:Restoring parameters from C:/Users/User/Raw data/PTT\model\PTT_Encoder-Decoder_model_fixed_1522514424.640014\model-20

Test index:  12779
Test input Words: <GO> 連假 好 無聊 聊聊 啊 因為 隔天 要 工作 所以 沒有 出去玩 今晚 想 去 看 潘恩 好 喜歡 休 傑克 曼 啊啊啊 想 揪 台 中 的 朋友 陪 我 一起 看 場次 如下 因為 我 有 優惠 票 一個 人 180 元 怕 你們 覺得 我 詐騙 所以 到 現場 跟 我 拿票 以後 再 跟 我 錢 最多 6 位 9 30 截止 想 跟 我們 一起 看 電影 的 請 私訊 我 喔 LINE <EOS>
Target Words: <GO> 台 中 老虎 城 一起 看 潘恩 <EOS>
Response Words: <GO> 今天 2 2 片 去 去 不 不 去 片 去 去 電影 電影 電影 電影 嗎 嗎 嗎 的 <EOS>

Test index:  7984
Test input Words: <GO> 大家 好 小弟 是 快要 畢業 的 115 碩 有 兩個 研替 offer 想 請問 前輩 們 公司 啟 碁 機器 學習 工程師 矽品 PIE 地點 竹科 中科 工時 8 8 00 住宿 租屋 租屋 薪資 N 14 獎金 N 16 N 為 GG 碩士 新人 價 啟 碁 的 職缺 與 我學 的 比較 相關 矽品 的 職缺 感覺 就是 進去 全部 重學 想 請問 前輩 們 哪個 職缺 或 公司 比較 有 前景 謝謝 <EOS>
Target Words: <GO> 研替 offer 請益 <EOS>
Response Words: <GO> 第 2 不 不 電影 電影 了 了 嗎 的 的 <EOS>

Test index:  5212
Test input Words: <GO> 想 請問 各位 因為 之前 有 投 履歷 跟 在 台積 電網 站 登錄 履歷 應徵 技術員 因為過 了 1 2 個 月 沒 消息 就 放棄 可是 今天 收到 一個 簡 訊 上面 積電 龍潭 科學園區 封裝 